In [ ]:
import pandas as pd
from math import sqrt
from datetime import datetime

In [ ]:
# read relevant data
stations_in_use_file = '../DATA/00_stations_in_use.csv'
stations_in_use = pd.read_csv(stations_in_use_file, delimiter=';')

In [ ]:
# function which calcultes the n nearest neigbors to a given station
def calc_nearest_neighbors(obs, station, number=3):
    obs = obs[obs.name != station[0]]
    result = []
    east = station[1]
    north = station[2]
    up = station[3]
    for row in obs.iterrows():
        cr = row[1]
        dist = sqrt((east-cr['east'])*(east-cr['east'])+(north-cr['north'])*(north-cr['north'])+(up-cr['up'])*(up-cr['up']))
        r = [dist, ':{}'.format(cr['name'])]
        result.append(r)
    result.sort(key=lambda x: x[0])
    return result[0:number]

In [ ]:
# calc n nearest neighbors for each station - create peudo distance-matrix
neighbors = []
n = 50
for idx, row in stations_in_use.iterrows():
    station = ['{}'.format(row['name'])]
    station.append(calc_nearest_neighbors(stations_in_use, row, number=n))
    neighbors.append(station)
    
header = 'name;east;north;up;timestamp;value'
for i in range(n):
    header += ';dist_{};value_{}'.format(i+1, i+1)

In [ ]:
# read all valid observations from previous progress
with open('../DATA/03_valid_observations.csv', 'r', encoding='utf-8') as file_in:
    lines = file_in.readlines()
lines = [x.strip() for x in lines]
del lines[0]

In [ ]:
# combine calculated neighbors and valid observations
with open('../DATA/04_station_nearest_stations-names.csv', 'a') as file_out:
    file_out.write('{}\n'.format(header))
    l = len(lines)
    cc = 1
    pc = 1
    for line in lines:
        # compute progress
        perc = (cc/l)*100
        if perc > pc:
            print('{0}: {1:.1f}%'.format(datetime.now(), perc))
            pc+=1
        cc+=1

        
        split = line.split(';')
        for row in neighbors:
            if row[0] == split[0]:
                str1 = ''.join(str(e) for e in row[1])
                str1=str1.replace("'"", '').replace('][', ';').replace('[', '').replace(']', '').replace(', :', ';')
                file_out.write('{0};{1}\n'.format(line, str1))
                break